## Testing: &thinsp; featues of dataset

In [16]:
import xarray as xr
import numpy as np
from os.path import expanduser
home = expanduser("~")
import skimage.measure as skm
import matplotlib.pyplot as plt
import cartopy

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [17]:
def plot_scene(scene, cmap='Reds', title='', vmin=None, vmax=None, fig_width=20 ,fig_height=10):
    projection = cartopy.crs.PlateCarree(central_longitude=180)
    lat = scene.lat
    lon = scene.lon

    f, ax = plt.subplots(subplot_kw=dict(projection=projection), figsize=(fig_width, fig_height))
    scene.plot(transform=cartopy.crs.PlateCarree(), cbar_kwargs={'orientation': 'horizontal','pad':0.125, 'aspect':50,'fraction':0.055}, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.add_feature(cartopy.feature.COASTLINE)
    ax.set_extent([lon[0], lon[-1], lat[0], lat[-1]], crs=cartopy.crs.PlateCarree())
    ax.set_title(title)
    ax.set_xticks([-180, -90, 0, 90, 180])
    ax.set_xticklabels([0, 90, 180, 270, 360])
    ax.set_yticks([-20, 0, 20])
    plt.tight_layout()

In [18]:
def plot_timeseries(y, variable_name='', series_type ='', title='', xmin = None, ymin = None):
    plt.figure(figsize=(25,5))
    plt.plot(y)
    plt.axhline(y=y.mean(dim='time'), color='k')
    plt.title(title)
    plt.ylabel(variable_name)
    plt.xlabel(series_type)
    plt.ylim([xmin,ymin])

In [19]:
def calc_sMean(var):
    aWeights = np.cos(np.deg2rad(var.lat))
    return var.weighted(aWeights).mean(dim=('lat','lon'))

In [20]:
import os
def save_file(dataset, folder, filename):
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + filename

    if os.path.exists(path):
        os.remove(path)    
    
    dataset.to_netcdf(path)

In [21]:
model = 'GFDL-CM3'
experiments =[
    'historical',
    # 'rcp85'
    ]
experiment = experiments[0]

variables = [
    'precip',
    # 'tas',
    # 'hus',
    # 'hur',
    # 'wap',
    # 'clouds',
    # 'pw',
    # 'wap500',
    # 'clMax',
]

obs = 'GPCP'
files = [
    'precip_raw',
    # 'precip_orig',
    # 'precip'
]

# ds = xr.open_dataset(home + '/Documents/data/cmip5/ds/' + model + '_orig/' + model + '_' + variables[0] + '_' + experiment + '.nc')[variables[0]]
# ds = xr.open_dataset(home + '/Documents/data/cmip5/ds/' + model + '/' + model + '_' + variables[0] + '_' + experiment + '.nc')[variables[0]]
ds = xr.open_dataset(home + '/Documents/data/obs/ds' + '/' + obs + '_' + files[0] + '.nc')
ds

<xarray.Dataset>
Dimensions:  (lat: 71, lon: 360, time: 9374)
Coordinates:
  * lat      (lat) float32 -35.0 -34.0 -33.0 -32.0 -31.0 ... 32.0 33.0 34.0 35.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1996-10-01 1996-10-02 ... 2022-05-31
Data variables:
    precip   (time, lat, lon) float32 ...

In [25]:
ds

<xarray.Dataset>
Dimensions:  (lat: 71, lon: 360, time: 9374)
Coordinates:
  * lat      (lat) float32 -35.0 -34.0 -33.0 -32.0 -31.0 ... 32.0 33.0 34.0 35.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1996-10-01 1996-10-02 ... 2022-05-31
Data variables:
    precip   (time, lat, lon) float32 ...

In [26]:
precip = ds.precip.sel(lat = slice(-30,30), time = slice('1998','2021'))
precip

<xarray.DataArray 'precip' (time: 8766, lat: 61, lon: 360)>
[192501360 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -30.0 -29.0 -28.0 -27.0 -26.0 ... 27.0 28.0 29.0 30.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2021-12-31
Attributes:
    long_name:      NOAA Climate Data Record (CDR) of Daily GPCP Satellite-Ga...
    standard_name:  lwe_precipitation_rate
    units:          mm/day
    valid_range:    [  0. 100.]
    cell_methods:   area: mean time: mean
    coordinates:    time latitude longitude

In [22]:
# var1=
# var2=

In [23]:
# plot_scene(var1.isel(time=0).sel(plev=850e2), 'Reds')
# plot_scene(var1.mean(dim ='time').sel(plev=850e2), 'Reds')
# plot_scene(var2.isel(time=0), 'Reds')
# plot_scene(var2.mean(dim ='time'), 'Reds')

In [24]:
# plot_timeseries(calc_sMean(var1))